In [1]:
%matplotlib inline

from matplotlib import pyplot as plt
from crn_utils import SimulationOptions, run_simulation
import numpy as np

simulation_options = SimulationOptions()
simulation_options.set_java_path("/home/harry/.jdks/openjdk-19.0.1/bin/java")
simulation_options.set_max_time(2000)

In [ ]:
simulation_options.set_repeats(100)
simulation_options.set_volumes([int(volume) for volume in np.logspace(2.0, 4.0, 10)])
simulation_options.net_from_file("./nets/tri-molecular.json")
simulation_options.use_temp_file(True)

data_tri = run_simulation(simulation_options)
places = data_tri.columns.difference(["repeatNum", "time", "volume"])
data_tri = data_tri.sort_values(["volume", "repeatNum", "time"], ascending = [True, True, True])
data_tri["isFinal"] = data_tri.time.eq(data_tri.groupby(['volume','repeatNum']).time.transform('max'))


In [ ]:
t_end_av = []
t_end_err = []
n_vals = []

for n in data_tri["volume"].unique():
    t_end = []
    for repeat in data_tri["repeatNum"].unique():
        t_end.append(data_tri[data_tri["volume"].eq(n) & data_tri["repeatNum"].eq(repeat)]["time"].max())
    t_end_av.append(np.mean(t_end))
    t_end_err.append(np.std(t_end))
    n_vals.append(n)

plt.xscale("log")
plt.xlabel("$n$")
plt.ylabel("time (s)")
plt.scatter(n_vals, t_end_av)
plt.errorbar(n_vals, t_end_av, t_end_err)
plt.show()

In [ ]:
x_vol = []
y_correctness = []
for volume in data_tri["volume"].unique():
    x_vol.append(volume)
    correct = data_tri[data_tri["volume"].eq(volume) & data_tri["isFinal"] & data_tri["X"].eq(0)].size
    incorrect = data_tri[data_tri["volume"].eq(volume) & data_tri["isFinal"] & data_tri["X"].ne(0)].size
    y_correctness.append(correct/(correct+incorrect))

plt.plot(x_vol, y_correctness)
plt.xscale('log')
plt.show()

In [ ]:
for place in places:
    for volume in data_tri["volume"].unique():
        plt.step(data_tri[data_tri["volume"].eq(volume)]["time"], data_tri[data_tri["volume"].eq(volume)][place], where='post', label="{}-{}".format(place, volume))
        plt.xlabel("time (s)")
        plt.ylabel("$n_{Ra}$")
        plt.title(place+"-"+str(volume))
        plt.show()


In [ ]:
simulation_options.net_from_file("./nets/double-b.json")
simulation_options.set_repeats(1000)
simulation_options.set_volumes([100, 500, 1000])
simulation_options.use_temp_file(False)

data_double_b = run_simulation(simulation_options)
places = data_double_b.columns.difference(["repeatNum", "time", "volume"])
DATA500 = data_double_b[data_double_b["volume"] == 1000]


In [ ]:

fig, axs = plt.subplots(len(places), sharex=True)
fig.suptitle("random sample")

row = 0
for place in places:
    for repeat in DATA500["repeatNum"].unique():
        axs[row].step(DATA500[DATA500["repeatNum"] == repeat]["time"], DATA500[DATA500["repeatNum"] == repeat][place], where='post', label="{}-{}".format(place, repeat))
        axs[row].set_xlabel("time (s)")
        axs[row].set_ylabel("$n_{Ra}$")
        axs[row].set_title(place)
    row += 1


In [ ]:
t_end = [DATA500[DATA500["repeatNum"].eq(r)]["time"].max() for r in DATA500["repeatNum"].unique()]

plt.title("Histogram of final states")
plt.xlabel("$t_{final}$")
plt.ylabel("frequency")
plt.hist(t_end, bins=101);